In [1]:
from datasets import load_dataset
dataset = load_dataset("jerteh/SrpKorNews")

In [2]:
print(dataset['train']['text'][0])
print(type(dataset['train']['text']))
print(len(dataset['train']['text']))

<s>Mečka na usijanom limenom krovu</s><s>Piše: Dušan Vidaković Kada je austrijski konzul u Nici Emil Jelinek 1901. naručio od nemačkog Dajmlera, odjednom, čak 36 automobila, čija je ukupna cena premašila fantastičnih 550 hiljada zlatnih maraka, imao je samo jedan uslov: da sva ta kola nose ime njegove ćerke.</s><s>Ljudi iz kanstatske firme, naravno, nisu imali ništa protiv, a istorija automobilizma je imala sreću da je Jelinekovoj mezimici kum dao (pogotovo za Špance) vrlo graciozno ime - Mercedes, što na jeziku konkvistadora znaci ljupkost, umilnost, prijatnost.</s><s>Jelinekova porudžbina značila prekretnicu u poslovanju Dajmlera, a naziv Mercedes je preživeo i fuziju sa fabrikom Karla Benca, 1926. godine, održavši se do danas kao sinonim za kvalitetne, luksuzne i, nadasve, bezbedne automobile svojstvenih vozačima koji ne kriju brige u novčaniku.</s><s>Međutim, upravo legenda o blistavosti trokrake zvezde doživljava ovih dana ozbiljnu proveru zahvaljujući mercedesu A klase.</s><s>A s

In [3]:
texts = dataset['train']['text'][:70]
print(texts[2])
print(len(texts))

<s>Iskoristite poslednju šansu</s><s>Dok šefovi država sveta u Meksiku uzalud troše četiri dragocena dana pokušavajući da ubede javnost da saopštenjima za novinare mogu da reše svetske ekonomske probleme, i probleme životne sredine u Brazilu, evropski lideri zaista imaju poslednju šansu da odluče o svojoj sudbini.</s><s>Poslednju šansu, jer ako ne urade ništa, opstanak evra će ponovo, narednog leta, zavisiti od masivnih intervencija centralnih banaka.</s><s>A sve naše zemlje ubrzo će preplaviti ogroman talas depresije, pa će kao čepovi plutati po okeanu tržišta.</s><s>Poslednju šansu, jer još uvek shvataju da razdvojeni rizikuju propast; dok bi ujedinjeni dugo mogli da budu vodeća svetska sila, sa najvišim životnim standardom na planeti.</s><s>Da bi se ovo desilo, svako treba da krene u susret onom drugom.</s><s>Nemci moraju da shvate da je njihova trenutna snaga samo iluzija ako se Evropa raspadne.</s><s>I da imaju lični interes u tome da daju malo vremena drugima, kao i sebi, za svođ

In [4]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("jerteh/Jerteh-355")
model = AutoModelForMaskedLM.from_pretrained("jerteh/Jerteh-355")

/home/teodora/Development/SRBedding/.venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [5]:
from bertopic import BERTopic
from umap import UMAP

# Prepare embeddings
embeddings = tokenizer(texts)

# # Train BERTopic
# topic_model = BERTopic().fit(texts, embeddings)

# # Run the visualization with the original embeddings
# topic_model.visualize_documents(texts, embeddings=embeddings)

# # Reduce dimensionality of embeddings, this step is optional but much faster to perform iteratively:
# reduced_embeddings = UMAP(n_neighbors=10, n_components=2, min_dist=0.0, metric='cosine').fit_transform(embeddings)
# topic_model.visualize_documents(texts, reduced_embeddings=reduced_embeddings)
embeddings

In [ ]:
from bertopic import BERTopic
from transformers import AutoTokenizer, AutoModelForMaskedLM
from torch import LongTensor, no_grad
from scipy import spatial
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer# Defining model pipeline

umap_model = UMAP(n_neighbors= 5, n_components= 3, metric='cosine', random_state=42)
hdbscan_model = HDBSCAN(min_cluster_size = 2, metric ='euclidean', cluster_selection_method = 'eom', prediction_data=True)
vectorizer_model = CountVectorizer(min_df = 3, max_df = 0.8, ngram_range = (1, 2))

topic_model = BERTopic(

  # Pipeline models
  
  embedding_model=model,
  umap_model=umap_model,
  hdbscan_model=hdbscan_model,
  vectorizer_model=vectorizer_model,
  min_topic_size = 3,

  # Hyperparameters
  top_n_words=10,
  verbose=True
)

In [ ]:
# Train model
topics, probs = topic_model.fit_transform(texts)

# Show topics
topics_bert = topic_model.get_topic_info()
topics_bert

2024-08-12 17:05:13,579 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/3 [00:00<?, ?it/s]

2024-08-12 17:06:30,784 - BERTopic - Embedding - Completed ✓
2024-08-12 17:06:30,784 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-08-12 17:06:37,518 - BERTopic - Dimensionality - Completed ✓
2024-08-12 17:06:37,519 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-08-12 17:06:37,539 - BERTopic - Cluster - Completed ✓
2024-08-12 17:06:37,545 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-08-12 17:07:52,955 - BERTopic - Representation - Completed ✓


,Topic,Count,Name,Representation,Representative_Docs
0,-1,10,-1_font_font family_size_font size,"[font, font family, size, font size, korona, s...",[<s>Beograd noćas mora pasti</s><s>Deci koja b...
1,0,10,0_font_font family_font size_size,"[font, font family, font size, size, belex15, ...",[<s>O čemu smo govorili protekle godine</s><s>...
2,1,8,1_je vesić_beogradskoj berzi_indeksa_belex15,"[je vesić, beogradskoj berzi, indeksa, belex15...",[<s>Patrole komunalne milicije obišle više od ...
3,2,7,2_brnabić_premijerka_onomad_šešelj,"[brnabić, premijerka, onomad, šešelj, prvorođe...",[<s>Tadićev dvokorak</s><s>Dok se Aleksandar V...
4,3,6,3_ilirika_agrobanke_000 km_suv,"[ilirika, agrobanke, 000 km, suv, renault, vw,...",[<s>Mečka na usijanom limenom krovu</s><s>Piše...
5,4,5,4_koštunica_šešelj_zločin_šešelja,"[koštunica, šešelj, zločin, šešelja, kišjuhas,...",[<s>LjUBAV JE JEDINI SPAS</s><s>Sa vrsnim glum...
6,5,4,5_toyota_corolla_peugeot_vw,"[toyota, corolla, peugeot, vw, električnih aut...",[<s>Pravi med je skup:</s><s>Evo kakve namete ...
7,6,4,6_trgovanju_indeksa_beogradskoj berzi_promet od,"[trgovanju, indeksa, beogradskoj berzi, promet...",[<s>Radno vreme za praznike</s><s>U ponedeljak...
8,7,4,7_ličnost danas_koštunica_srebrenici_msp,"[ličnost danas, koštunica, srebrenici, msp, zl...",[<s>Vanredno stanje uma</s><s>Šta znači kad je...
9,8,4,8_je vesić_radojičić_font_beoinfo,"[je vesić, radojičić, font, beoinfo, vesić je,...","[<s>U gradu 130.000 turista</s><s>Danas, popod..."


In [ ]:
topic_model.visualize_documents(texts, reduced_embeddings=reduced_embeddings, 
                                hide_document_hover=True, hide_annotations=True)

In [35]:
topic_model.visualize_topics(custom_labels=True)

In [40]:
new_topics = new_topics = topic_model.reduce_outliers(texts, topics, strategy="embeddings")
topic_model.update_topics(texts, topics=new_topics)

ValueError: No outliers to reduce.

In [42]:
topic_model.visualize_topics(custom_labels=True)